In [67]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torch_data

In [ ]:
import pandas as pd
import numpy as np

regular = pd.read_csv("datasets/regular_season_totals_2010_2024.csv")
playoff = pd.read_csv("datasets/play_off_totals_2010_2024.csv")

def get_data(regular, playoff):
    totals = pd.concat([regular, playoff], ignore_index=True)
    totals["GAME_DATE"] = pd.to_datetime(totals["GAME_DATE"])
    totals = totals.sort_values("GAME_DATE").reset_index(drop=True)

    # победа дома
    totals["is_home"] = totals["MATCHUP"].str.contains("vs.")
    totals["HOME_WIN"] = ((totals["WL"] == "W") & totals["is_home"]).astype(int)

    # Начальные ELO
    teams = totals["TEAM_NAME"].unique()
    elo_ratings = {team: 1500 for team in teams}
    last_game_date = {team: None for team in teams}

    elo_list = []
    rest_days_list = []


    for _, row in totals.iterrows():
        team = row["TEAM_NAME"]
        is_home = row["is_home"]
        date = row["GAME_DATE"]

        # ELO
        elo = elo_ratings[team]
        elo_list.append(elo)

        # Rest
        last_date = last_game_date[team]
        rest_days = (date - last_date).days if last_date is not None else 7
        rest_days_list.append(rest_days)

        # updat ELO после игры
        score = 1 if (row["WL"] == "W" and is_home) or (row["WL"] == "L" and not is_home) else 0
        new_elo, _ = update_elo_single(elo_ratings[team], 1500, score)
        elo_ratings[team] = new_elo
        last_game_date[team] = date

    totals["ELO"] = elo_list
    totals["REST_DAYS"] = rest_days_list

    # Фичи
    features = [
        "PTS", "REB", "AST", "STL", "BLK", "TOV",
        "FGM", "FGA", "FG_PCT",
        "FG3M", "FG3A", "FG3_PCT",
        "FTM", "FTA", "FT_PCT",
        "OREB", "DREB", "PF", "PFD", "PLUS_MINUS",
        "ELO", "REST_DAYS"
    ]

    # Train / Test split по дате(чтобы не знал о будущих данных при shuffle )
    split_date = pd.Timestamp("2022-01-01")
    train_dataset = totals[totals["GAME_DATE"] < split_date]
    test_dataset = totals[totals["GAME_DATE"] >= split_date]

    X_train = train_dataset[features].to_numpy().astype(np.float32)
    y_train = train_dataset["HOME_WIN"].to_numpy().astype(np.int64)

    X_test = test_dataset[features].to_numpy().astype(np.float32)
    y_test = test_dataset["HOME_WIN"].to_numpy().astype(np.int64)

    return X_train, X_test, y_train, y_test

# Функция обновления ELO для одного матча
def update_elo_single(elo_a, elo_b, score_a, k=20):
    expected_a = 1 / (1 + 10 ** ((elo_b - elo_a) / 400))
    expected_b = 1 - expected_a
    new_a = elo_a + k * (score_a - expected_a)
    new_b = elo_b + k * ((1 - score_a) - expected_b)
    return new_a, new_b

X_train, X_test, y_train, y_test = get_data(regular, playoff)

print("Размер X_train:", X_train.shape)
print("Размер X_test:", X_test.shape)
print(X_test)

Размер X_train: (28824, 22)
Размер X_test: (6854, 22)
[[ 1.1600000e+02  4.8000000e+01  3.0000000e+01 ... -1.0000000e+00
   1.4890217e+03  1.0000000e+00]
 [ 1.1600000e+02  5.7000000e+01  2.6000000e+01 ... -4.0000000e+00
   1.4927789e+03  2.0000000e+00]
 [ 1.2300000e+02  4.0000000e+01  3.9000000e+01 ...  7.0000000e+00
   1.5750912e+03  4.0000000e+00]
 ...
 [ 1.2200000e+02  5.2000000e+01  2.1000000e+01 ...  3.8000000e+01
   1.4909344e+03  2.0000000e+00]
 [ 1.0600000e+02  5.1000000e+01  2.5000000e+01 ...  1.8000000e+01
   1.5505931e+03  3.0000000e+00]
 [ 8.8000000e+01  3.5000000e+01  1.8000000e+01 ... -1.8000000e+01
   1.5011953e+03  3.0000000e+00]]


In [94]:
xgb_model = xgb.XGBClassifier()
log_model = LogisticRegression()
svm_model = SVC(kernel="rbf")
forest_model = RandomForestClassifier(max_depth=15)
nn_model = nn.Sequential(
    nn.Linear(22,64, bias=False),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64,32, bias=False),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(32,2, bias=True))

In [88]:
xgb_model.fit(X_train, y_train)
print(classification_report(y_test, xgb_model.predict(X_test)))


              precision    recall  f1-score   support

           0       0.88      0.79      0.84      4928
           1       0.58      0.74      0.65      1926

    accuracy                           0.78      6854
   macro avg       0.73      0.76      0.74      6854
weighted avg       0.80      0.78      0.78      6854



In [96]:
forest_model.fit(X_train, y_train)
print(classification_report(y_test, forest_model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      0.77      0.83      4928
           1       0.57      0.80      0.67      1926

    accuracy                           0.78      6854
   macro avg       0.74      0.79      0.75      6854
weighted avg       0.82      0.78      0.79      6854



In [97]:
log_model.fit(X_train, y_train)
print(classification_report(y_test, log_model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.82      0.87      0.84      4928
           1       0.61      0.51      0.55      1926

    accuracy                           0.77      6854
   macro avg       0.71      0.69      0.70      6854
weighted avg       0.76      0.77      0.76      6854



d:\mine\code\2025\ml-ai\nba_predictiction\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [98]:
svm_model.fit(X_train, y_train)
print(classification_report(y_test, svm_model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.76      0.95      0.84      4928
           1       0.63      0.22      0.33      1926

    accuracy                           0.74      6854
   macro avg       0.69      0.58      0.58      6854
weighted avg       0.72      0.74      0.70      6854



In [99]:
from tqdm import tqdm
class Dataset(torch_data.Dataset):
 def __init__(self, data_x, data_y):
  super().__init__()
  self.data_x = torch.tensor(data_x, dtype=torch.float32)
  self.data_y = torch.tensor(data_y, dtype= torch.long)
  self.length = len(data_y)
 def __getitem__(self, index):
  return self.data_x[index], self.data_y[index]
 def __len__(self):
  return self.length
opt = optim.Adam(params=nn_model.parameters(), lr = 0.001, weight_decay=0.001)
loss_func = nn.CrossEntropyLoss(weight=torch.tensor([0.28, 0.72]))
d_train = Dataset(X_train, y_train)
train_data = torch_data.DataLoader(d_train, batch_size=32, shuffle=True)
epochs = 50
for _ in range(epochs):
 tqdm_data = tqdm(train_data)
 for x,y in tqdm_data:
  predict = nn_model(x)
  loss = loss_func(predict, y)
  opt.zero_grad()
  loss.backward()
  opt.step()
 tqdm_data.set_description(f"epochs : {_+1}")


100%|██████████| 901/901 [00:02<00:00, 356.65it/s]


In [101]:
d_test = Dataset(X_test, y_test)
test_data = torch_data.DataLoader(d_test, batch_size=len(d_test), shuffle=False)
x,y = next(iter(test_data))
predict = torch.argmax(nn_model(x), dim=1)
print(classification_report(y_test, torch.argmax(nn_model(x), dim=1)))

              precision    recall  f1-score   support

           0       0.98      0.70      0.82      4928
           1       0.56      0.97      0.71      1926

    accuracy                           0.78      6854
   macro avg       0.77      0.84      0.77      6854
weighted avg       0.86      0.78      0.79      6854



In [102]:
torch.save(nn_model.state_dict(),"model_params.tar")

У нас дисбаланс классов матчей с проигрышем дома (0) больше — 4928,чем побед дома (1)  — 1926. Поэтому acc сам по себе мало что говорит, нужно смотреть precision и recall для каждого класса, особенно для редкого класса

1. XGBoost

Accuracy: 0.78 — неплохо

Класс 0 (проигрыш дома): precision 0.88, recall 0.79  неплохо угадывает проигрыши

Класс 1 (победа дома): precision 0.58, recall 0.74  модель находит большинство побед, но много ложных срабатываний
F1 для победы 0.65 — средне.

2. Random Forest

Accuracy: 0.78 — как у XGBoost

Класс 0: precision 0.91, recall 0.77  чуть лучше угадывает проигрыши, чем XGB

Класс 1: precision 0.57, recall 0.80  recall выше, но precision чуть ниже  лучше ловит победы, но предсказывает их больше, чем на самом деле

F1 для победы 0.67 — чуть лучше XGB

3. Logistic Regression

Accuracy: 0.77 — немного ниже

Класс 0: precision 0.82, recall 0.87  хорошо угадывает проигрыши, но precision ниже  иногда «пропускает» проигрыши

Класс 1: precision 0.61, recall 0.51  плохо ловит победы, много промахов

F1 для победы 0.55 — хуже всех

4. SVM

Accuracy: 0.74 — самая низкая

Класс 0: precision 0.76, recall 0.95  почти все проигрыши поймала, но за счёт того, что часто предсказывает проигрыш

Класс 1: precision 0.63, recall 0.22  почти не находит победы дома  крайне плохой для редкого класса

F1 для победы 0.33 — ужасно

5. Neural Network (NN)

Accuracy: 0.78 — на уровне XGB и Random Forest

Класс 0: precision 0.99, recall 0.70 почти всегда, когда предсказывает проигрыш, оно верно, но часть проигрышей пропускает

Класс 1: precision 0.56, recall 0.98 почти все реальные победы находит, но много ложных

F1 для победы 0.71 — лучший результат среди всех

Что видно по сути

XGB и Random Forest примерно одинаковы по accuracy, но RF чуть лучше для побед дома  (0.67 vs 0.65)

Logistic Regression и SVM плохо справляются с редким классом (победа дома)

NN выделяется почти все победы дома находит (recall 0.98), и F1 для победы выше всех (0.71). Precision чуть ниже, но это нормально при дисбалансе

Лучшая модель — Neural Network.

Почему:

Она лучше всего находит победы дома, что важнее при дисбалансе

F1 для побед выше всех, значит баланс между precision и recall лучше, чем у других моделей.

Accuracy примерно как у XGB/RF, но NN даёт лучшее качество предсказания редкого класса, а это ключевой момент